[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xiptos/is_notes/blob/main/model_sel.ipynb)

# Model Selection

The task of producing the most suitable predictive model from a dataset has many nuances. In particular, there are two aspects that need to be defined experimentally before starting to adjust a model:

* At first, it may not be clear which learning algorithm is best suited to be used in a given situation.

* In addition, each learning algorithm has its own set of *hyperparameters*, which are the parameters of the algorithm itself, to counter the concept of *parameter* of the ML model to be adjusted.

Model selection is the process of selecting one of different learning algorithms (e.g., SVM, decision tree learning, logistic regression, etc.) or, once the learning algorithm is selected, choosing between different values ​​of hyperparameters (e.g., in polynomial regression, deciding which polynomial degree to use during training).

For a given prediction task, the choice of the most appropriate learning algorithm (and its respective hyperparameter values) is made based on some *estimate* of the generalization error obtained on unused data during training. In other words, different candidate models are evaluated during model selection. The model that has the best predictive performance (according to some pre-established assessment measure) is selected as a result of the model selection process.

# Two-way holdout method

In classification or regression tasks, it is very common to split the available observations (original dataset ) in two datasets: 

* training dataset: used for training a model M
* test dataset: used to estimate the *generalization error* of M

This splitting method is commonly named *two-way holdout method*, or simply *holdout method*. This two-way split is illustrated in the image below ([source](https://www.datavedas.com/holdout-cross-validation/)).

![alt text](https://www.datavedas.com/wp-content/uploads/2018/04/image001-768x368.jpg)



In Scikit-Learn, we can apply the holdout method by using the `train_test_split` function. This function receives as input a data matrix $X$ and response vector $y$.

The following image illustrates the effect of applying the function `train_test_split`. As a result, two data matrices and two response vectors are created. 

![alt text](https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1543836883/image_6_cfpjpr.png)

Please notice that, each time `train_test_split` is called, it randomly selects which examples to put in the training and test datasets.

## Three-way holdout method

The application of the two-way holdout method can lead to a misleading estimate of the generalization error if the test data is also used for model selection.

Since the test dataset should ideally be used only to produce an estimate of the generalization error **after** training, it is common (at least for relatively large data sets) to further divide the training set into two subsets. The result of this is the production of three datasets, as described below.

* Training set: used to adjust the parameters for several candidate models.

* Validation set: used to adjust the hyperparameters of the ML algorithm. The predictive performance on this set is used for model selection. i.e., choose the best candidate model.  

* Test set: used once to obtain an estimate of the generalization error for the chosen candidate model.

The image below ([source](https://www.datavedas.com/holdout-cross-validation/)) illustrates the three-way split of the original dataset.

![alt text](https://www.datavedas.com/wp-content/uploads/2018/04/image003.jpg)

The image below ([source](https://datascience.stackexchange.com/questions/52632/cross-validation-vs-train-validate-test)) summarizes the model selection process for a given learning algorithm using the three data sets described above. Initially, the original data set is divided into two subsets, training and testing. Then, the training set is again divided into two, which generates the training, validation and test sets. After that, for each desired combination of hyperparameter values, a model is adjusted on the training set, and its performance on the validation set is computed. In this way, for each of the $ q $ combinations of hyperparameter values, $ q $ performance estimates are obtained. The predictive model corresponding to the combination of hyperparameter values ​​that produces the best performance estimate is selected. Finally, the generalization error estimate for this selected model is produced using the test set.

![alt text](https://i.stack.imgur.com/osBuF.png)

The procedure illustrated in the figure above can be performed once for each of the different learning algorithms candidates in a supervised learning task.

### Split proportions

Reserving  a test dataset to compute an estimate of the generalization error has the negative side effect of retaining potentially valuable information that the learning algorithm could benefit from. Therefore, choosing the size of the test set  involves a trade-off: allocating a very large test set can retain a lot of information that could be used to improve model fit during training. On the other hand, the smaller the test set, the more inaccurate the generalization error estimate is.

In most practical cases, the most used divisions are 60:40,
70:30 or 80:20, depending on the size of the initial data set.

However, for large data sets, 90:10 or even 99: 1 may be appropriate, since 10% (or even 1%) of a very large sample can still be statistically significant.

A common practice is to re-train the estimator on the entire data set (i.e., rejoining the training and test sets), in order to improve predictive performance of the resulting final model.

## $k$-fold Cross-validation

There are two disadvantages of the holdout method:

* the performance estimate can be very sensitive to the way used to subdivide the training set into the training and validation sets: this estimate will vary for different data samples. Repeating the holdout validation on different subsets of the data would provide a better estimate of the algorithm's performance.

* a significant part of the data is not used for training the model. For example, if the split ratio is 50%, half of the data set does not contribute to model training! This can cause problems, especially if the initial data set is small.

A more robust technique for performing model selection is *k-fold cross-validtion*. This technique corresponds to applying the holdout method $k$ times over $k$ different configurations of training and validation subsets.

The following figure ([source](https://sebastianraschka.com/faq/docs/evaluate-a-model.html)) illustrates $k$-fold cross validation. This is an iterative technique. In the first iteration, the training set is subdivided into $k$ disjunct sets (in the figure, $k = 10$). Then, $k-1$ of these subsets are used to fit the predictive model. Once the model has been adjusted, its error is estimated in the examples of the subset that was not used during the adjustment, which produces the value $E_1$ in the figure. In the second iteration, another fold is selected to compute the estimate and again a model is adjusted, which allows to compute another estimate of the error, $E_2$. In all, $k$ iterations are performed, which results in $k$ performance estimates. Finally, a more robust (i.e., less biased) estimate of the model error is obtained by computing the average of the previously produced $k$ estimates.

![alt text](https://sebastianraschka.com/images/faq/evaluate-a-model/k-fold.png)

Keep in mind that, as k increases:
* Bias in the estimation of the generalization error decreases;
Computational cost also increases.
* Empirical evidence shows that k=10 is a good value for datasets of moderate size.
* For large sized datasets, k can be safely decreased.


### Function cross_val_score

In the following example, the $k$-fold cross-validation technique is applied, with $k = 2$. The result is two performance estimates, which can be combined (for example, taking the average) to obtain a better measure of the model's generalization performance. This specific form of cross-validation is double cross-validation: we split the original data into two subsets and use each one as a validation set.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Configures k-NN with 1 neighbor
model = KNeighborsClassifier(n_neighbors=1)

iris = load_iris()
X = iris.data
y = iris.target

# divide the original dataset in 2 equal parts
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, train_size=0.5)

# ajusta e avalia DOIS modelos

y2_model = model.fit(X1, y1).predict(X2)

y1_model = model.fit(X2, y2).predict(X1)

accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

Implementing $k$-fold cross validation using the approach illustrated in the previous example can be time consuming for greater values of $k$.

Scikit-Learn provides the function [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). This function makes it easier to apply $k$-fold cross validation. 

It returns a NumPy array with an accuracy value for each fold. Notice that, for each fold, the training and tests sets will be different. Therefore, different accuracy values for each fold will be produced. Also notice that we can compute the mean and standard deviation of the resulting accuracies.

The following example illustrates the application of the function cross_val_score to the case $k = 10$.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=10)
print('Scores in each fold: ', scores)
print('Average score: ', scores.mean())
print('Std deviation of scores: ', scores.std())

Scikit-Learn implements several cross-validation schemes that are useful in particular situations; these are implemented through iterators in the cross_validation module. In practice, we have to define the iterator through the `cv` parameter.

As an example, we might want to go to the extreme case where the number of subsets is equal to the number of examples contained in the training set. In this case, in each iteration, a model is fitted using all but one example, and accuracy is measured against the only example not used during training. This type of cross-validation is known as **leave-one-out cross-validation**.

The example below illustrates the use of this particular case leave-one-out cross-validation. Since there are 150 samples, the leave-one-out cross-validation produces 150 estimates (1.0 indicates a successful forecast, and 0.0 indicates an unsuccessful one).

In [ ]:
from sklearn.model_selection import LeaveOneOut

scores = cross_val_score(model, X, y, cv=LeaveOneOut())
scores

The average of these 150 values ​​corresponds to the estimated error rate of the model produced:

In [ ]:
print('Average score: ', scores.mean())
print('Std deviation of scores: ', scores.std())

## Stratified k-fold cross-validation

In the context of the classification task, an unbalanced presents as unequal proportion of examples for each class. See the image below ([source](https://www.researchgate.net/publication/306376881_Survey_of_resampling_techniques_for_improving_classification_performance_in_unbalanced_datasets)) for an example.

![alt text](https://miro.medium.com/max/698/1*cd6AorHoJYMFyj7IZd2nOg.png)

For unbalanced datasets, if the vanilla version of $k$-fold cross-validation is used, it can be the case that one or more produced folds end up with zero examples of a particular class. In such situations, it is better to use *stratified $k$-fold cross-validation*. In stratified $k$ cross-validation, class proportions are preserved in each of the $k$ subsets, so that each one is representative of the class proportions in the training data set. The image below (source) illustrates how stratified $k$-fold crors-validation works.

![alt text](https://miro.medium.com/max/562/0*QKJTHrcriSx2ZNYr.png)

In Scikit-Learn, the `StratifiedKFold` class implements the stratified cross-validation method. See the following example, that applies stratified cross-validation to a toy dataset with two classes. Notice that the proportions os classes in each fold is approximately the same as the original dataset.

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

# data matrix
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [5, 6], [7, 8], 
              [9, 10], [9, 10], [11, 12], [11, 12],
              [1, 2], [3, 4], [5, 6], [7, 8], [4, 5], [5, 6], [7, 8], [4, 5]])

# response vector
y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

skf = StratifiedKFold(n_splits=2, shuffle=True)
skf.get_n_splits(X, y)

print(skf)  

# print the indices of examples in each fold
for train_index, test_index in skf.split(X, y):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = X[train_index], X[test_index]
   y_train, y_test = y[train_index], y[test_index]

# Hyperparameter optimization

In contrast to the model parameters that are learned during training, the values ​​of the hyperparameters are set before the training of a model begins. Some examples of hyperparameters are given below.

* the learning rate in logistic regression and in linear regression, 
* the height of trees in decision tree learning,
* the value of $k$ in $k$-NN, 
* the value of $k$ in $k$-means, 
* the values of $\epsilon$ and $\operatorname{minPoints}$ in DBSCAN,
* the regularization term in linear regression,
* the polynomial degree in polynomial regression.


Hyperparameters can be considered as configurations of a learning algorithm. In general, the ideal settings for the algorithm to generate a suitable model for one dataset may not be the same for another dataset. 

*Hyperparameter optimization* (aka *hyperparameter tuning*) is the procedure of defining appropriate values for the hyperparameters of a given learning algorithm. The general approach is to generate several combinations of hyperparameter values. Then, a model is trained for each combinatation. Finally, the best combination (according to some evaluation measure applied on the candidate models) is chosen.

When the learning algorithm generates an estimator (either a classifier or a regressor), there are two main strategies to *automatically* tune its hyperparameters: *Grid Search* and *Random Search*. Both strategies generate several hyperparameter combinations. Then, for each combination, an estimator is trained a model (on the training set) and evaluateed (on the validation set). The difference between these two strategies is in the way the several combinations are generated, as described below.

**Grid search** first defines a grid of hyperparameter combinations. The amount of hyperparameters defines the number of dimensions in the grid. Then, each combination (a point in the grid) ​​is used to produce a model.

The image below ([source](https://www.researchgate.net/publication/271771573_TuPAQ_An_Efficient_Planner_for_Large-scale_Predictive_Analytic_Queries/figures?lo=1)) illustrates the GS procedure for a learning algorithm with two hyperparameters. The star represents the combination of hyperparameters that gives the best model. as measured in the validation set. The red dot in picture on the left represents the first combination test by GS. The picture on the center represents the second combination. The picture on the right illustrates all the tested combinations in the 2-dimensional grid.

![texto alternativo](https://www.researchgate.net/profile/Michael_Jordan13/publication/271771573/figure/fig4/AS:668513593217027@1536397469229/Illustration-of-naive-grid-search-for-a-single-model-family-with-two-hyperparameters_W640.jpg)

**Random search** selects random combinations of hyperparameter values ​​(within preconfigured ranges of values for each hyperparameter) to train and evaluate the candidate models. Instead of trying all possible combinations of values, RS performs a pre-defined number of iterations, testing a random combination of hyperparameters ​​in each iteration.

Both GS and RS can be very computationally expensive. For example, searching for 20 different values ​​for each of 4 hyperparameters would require 160,000 k-fold cross-validation runs. If $k = 10$ then 1,600,000 model adjustments and 1,600,000 evaluations should be performed

The image below ([source](https://community.alteryx.com/t5/Data-Science/Hyperparameter-Tuning-Black-Magic/ba-p/449289)) illustrates the difference between GS and RS. Think of a learning algorithm with just two hyperparameters. This way, each combination of its hyperparameters is a pair of numbers. Suppose one of these hyperparameters (x-axis) has more influence than the other (y-axis) on the predictive performance of the generated models. The plot on the left shows several of these pairs organized in a grid. In total, there are thirty combination in this grid. The picture on the right shows other combinations of pairs; this times these combinations where randomly selected. Notice that RS has the potential to explore more promising combinations than GS.

![texto alternativo](https://pvsmt99345.i.lithium.com/t5/image/serverpage/image-id/74545i97245FDAA10376E9/image-size/large?v=1.0&px=999)

In Scikit-Learn, the classes [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) implement GS and RS, respectively.

### Class GridSearchCV

Scikit-Learn provides the class `GridSearchCV` to perform grid search.

The following code block presents an example of using the `GridSearchCV` class to find the ideal polynomial model for a data set. In this example, a three-dimensional grid of hyperparameters is explored:

* the polynomial degree,
* a boolean indicator that indicates whether the linear coefficient should be adjusted,
* an indicator (boolean) indicating whether the data should be normalized.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

def polinomial_regression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),LinearRegression(**kwargs))

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False]}

def make_data(N, err=1.0, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

In [ ]:
X, y = make_data(40)
grid = GridSearchCV(polinomial_regression(), param_grid, cv=7)
grid.fit(X, y);

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # plot formatting

X_test = np.linspace(-0.1, 1.1, 500)[:, None]

plt.scatter(X.ravel(), y, color='black')
axis = plt.axis()
for degree in [1, 3, 5]:
    y_test = polinomial_regression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label='degree={0}'.format(degree))
plt.xlim(-0.1, 1.0)
plt.ylim(-2, 12)

In [ ]:
grid.best_params_

In [ ]:
import matplotlib.pyplot as plt

model = grid.best_estimator_

plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = model.fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim);

### Class RandomizedSearchCV

This allows you to explicitly control the number of parameter combinations that are attempted. The number of search iterations is defined based on time or resources. Scikit Learn offers the RandomizedSearchCV function for this process.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from timeit import default_timer as timer
from sklearn.svm import LinearSVC

def linear_SVC(x, y, param, kfold):
    param_grid = {'C':param}
    k = StratifiedKFold(n_splits=kfold)
    grid = GridSearchCV(LinearSVC(dual=False), param_grid=param_grid, cv=k, n_jobs=4, verbose=1)

    return grid.fit(x, y)

def Linear_SVC_Rand(x, y, param, kfold, n):
    param_grid = {'C':param}
    k = StratifiedKFold(n_splits=kfold)
    randsearch = RandomizedSearchCV(LinearSVC(dual=False), param_distributions=param_grid, cv=k, n_jobs=4,
                                    verbose=1, n_iter=n)

    return randsearch.fit(x, y)

from sklearn.datasets import load_iris
iris=load_iris()
X=iris.data
y=iris.target

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


start = timer()
param = [i/1000 for i in range(1,1000)]
param1 = [i for i in range(1,101)]
param.extend(param1)

#progress = progressbar.bar.ProgressBar()
clf = Linear_SVC_Rand(x=x_train, y=y_train, param=param, kfold=3, n=100)

print('LinearSVC:')
print('Best cv accuracy: {}' .format(clf.best_score_))
print('Test set score:   {}' .format(clf.score(x_test, y_test)))
print('Best parameters:  {}' .format(clf.best_params_))
print()

duration = timer() - start
print('time to run RandomizedSearchCV: {}' .format(duration))


print('########')

#high C means more chance of overfitting

start = timer()
param = [i/1000 for i in range(1,1000)]
param1 = [i for i in range(1,101)]
param.extend(param1)

clf = linear_SVC(x=x_train, y=y_train, param=param, kfold=3)

print('LinearSVC:')
print('Best cv accuracy: {}' .format(clf.best_score_))
print('Test set score:   {}' .format(clf.score(x_test, y_test)))
print('Best parameters:  {}' .format(clf.best_params_))
print()

duration = timer() - start
print('time to run GridSearchCV: {}' .format(duration))

In [ ]:

import numpy as np

from time import time
from scipy.stats import randint as sp_randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

# get some data
digits = load_digits()
X, y = digits.data, digits.target

# build a classifier
clf = RandomForestClassifier(n_estimators=20)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv=5)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

## Model selection: practical tips

* It is appropriate to start with a simple algorithm that can be used quickly.

* Use learning curves to decide for more (or less) data, more (or less) attributes, etc.

* Another technique used in model selection is *error analysis*, which corresponds to manually checking the examples (in the validation set) that the algorithm has erred. It is sometimes possible to detect a pattern of systematic error in these examples.

* The motivation for using k-fold cross-validation is to be able to use a good amount of test or validation data, without significantly reducing the training data set. In situations where there is enough data to have a good-sized training data set, in addition to reasonable amounts of test and validation data, holdout cross validation can be used.


# Case Study: Housing dataset


In this case study, we will use the *Housing* dataset to practice some concepts related to hyperparameter optimization. This dataset contains information about residences in Boston/USA. There are 506 samples and 13 variables in this dataset. This dataset is used to train regressors, in which the objective is to predict the value of home prices (MEDV feature) using the predictive features provided. This dataset can be loaded with the Scikit-Learn library, according to the code block below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

from sklearn.datasets import fetch_california_housing
cal_dataset = fetch_california_housing()

print(cal_dataset.keys())
print(cal_dataset['DESCR'])

cal = pd.DataFrame(cal_dataset.data, columns=cal_dataset.feature_names)
cal['MEDV'] = cal_dataset.target
cal.head()

**(1)** 

a) Perform the split of the Housing dataset into training and test subsets using holdout strategy. Set aside 80% of the data for training. After that, build a OLS regression model with that split, and evaluate the MSE in the test dataset.

b) Repeat the previous experiment, this time using $k$-fold cross validation instead of the hold-out strategy. Use $k = 5$.

**(2)** Consider the following code block, which is intended to perform Grid Search on the `Housing` dataset. The `param_grid` parameter tells Scikit-Learn to first evaluate all 3 × 4 = 12 combinations of` n_estimators` and `max_features` with hyperparameters specified in the first dict (don't worry about what these hyperparameters mean for the moment), and then try all 2 × 3 = 6 combinations of hyperparameter values ​​in the second dict, but this time with the `bootstrap` hyperparameter set to` False` instead of `True` (which is the default value).
In short, GS will explore 12 + 6 = 18 combinations of hyperparameter values ​​for the `RandomForestRegressor` algorithm.

  a) Based on the value of $k$ defined to perform the cross-validation in the code below, how many models will be adjusted during GS?

  b) Complete the code given below to tune the hyperparameters using Grid Search. What is the RMSE value for the best model?
  
  c) Use Random Search with 18 iterations. Is there a change in the  produced best combination of hyperparameter values, when compared to Grid Search? What is the new RMSE value?

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error')
grid_search.fit(X, y)

To load the data, the `load_boston` function is imported. To standardize the data, the normalization function was used, which places the data with $ \ mu = 0 $ and $ \ sigma ^ 2 = 1 $

In [ ]:
import pandas as pd   
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import normalize

cal_dataset = fetch_california_housing()

cal = pd.DataFrame(normalize(cal_dataset.data), columns=cal_dataset.feature_names)
cal['MEDV'] = cal_dataset.target
cal.head()

### Answer: 1a

We can use the function `train_test_split` to create a training dataset with  80% of the examples, as requested.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = cal.drop(columns=['MEDV'])
y = cal['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.8, 
                                                    random_state=42)

print(X_train.shape)
print(X_test.shape)

reg = LinearRegression()

reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)

print('MSE:', mean_squared_error(y_test, y_pred))

### Answer: 1b

For the k-fold validation, five data divisions were performed between training and testing using the cross_value function with $ MSE $ validation metric

In [ ]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression

X = cal.drop(columns=['MEDV'])
y = cal['MEDV']

reg = LinearRegression()

scores = cross_validate(reg, X, y, cv=5, scoring='neg_mean_squared_error')

print(scores)
print(-scores['test_score'].mean())

### Answer: 2a

In total, 90 models will be trained by Grid Search as there are 18 parameters that will be combined and trained 5 times each

### Answer: 2b
The code below performs the training using Grid Search

In [ ]:
X,y = fetch_california_housing(return_X_y=True, as_frame=True)
housing = pd.concat([X,y], axis=1)
housing.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

housing_prepared = scaler.fit_transform(housing.drop(columns='MedHouseVal'))
housing_labels = housing.loc[:,'MedHouseVal'].values

In [ ]:
housing_labels

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error')
%time grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
import numpy as np

rmse = np.sqrt(-grid_search.best_score_)

rmse

### Answer: 2c

The code below performs the training using Random Search


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()
random_search = RandomizedSearchCV(forest_reg, 
                                   param_grid, 
                                   cv = 5,
                                   scoring='neg_mean_squared_error',
                                   n_iter = 18,
                                   verbose = 2)
%time random_search.fit(housing_prepared, housing_labels)

In [ ]:
random_search.best_params_

In [ ]:
import numpy as np
rmse = np.sqrt(-random_search.best_score_)
rmse

# Case Study: Wine Quality dataset

Let us see another case study of applying hyperparameter optimization using the Wine Quality dataset.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read in and create target
wine_data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',sep=';')
wine_data['high_quality'] = [1 if x >=7 else 0 for x in wine_data['quality']]

# Split data
X = wine_data.drop(['quality'], axis=1)
y = wine_data['high_quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read in and create target
wine_df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',sep=';')

wine_df.head()

In [ ]:
wine_df.info()

In [ ]:
wine_df.describe()

In [ ]:
# Split data
X = wine_df.drop(['quality'], axis=1)
y = wine_df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Create parameter grid
param_grid = {'C': [.01, .1, 1, 10, 100, 1000, 10000],
                  'gamma': [0.0001, 0.001, 0.01, .1, 1, 10, 100, 1000]}

# Create grid search 
grid_search = GridSearchCV(SVC(),
                           param_grid, 
                           cv=5)
# Fit grid search
%time grid_search.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Pull results from grid search
results = pd.DataFrame(grid_search.cv_results_)
hm_data = results.pivot(index='param_C', 
                   columns='param_gamma',
                   values='mean_test_score')

# Plot heatmap
fix, ax = plt.subplots(figsize=(9, 6))

ax = sns.heatmap(hm_data,
                 annot=True,
                 cmap='RdBu',
                 linecolor='white')

ax.set_title('Hyperparameter Gridsearch', fontsize=16)
ax.set_ylabel('C Parameter', fontsize=12)
ax.set_xlabel('Gamma Parameter', fontsize=12)

plt.show()

In [ ]:
# Print best parameters and best score
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

### Best parameters: {'C': 100, 'gamma': 0.0001}
### Best cross-validation score: 1.00

# Plug best parameters into the model, train it, and test on held out data
best_svc = SVC(**grid_search.best_params_).fit(X_train, y_train)
best_svc.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = best_svc.predict(X_test)

print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Create parameter grid
param_grid = {'C': [100, 500, 1000, 5000, 10000],
                  'gamma': [0.001/3, 0.001/2, 0.001, 0.001*2, 0.001*3]}

# Create grid search 
grid_search = GridSearchCV(SVC(),
                           param_grid, 
                           cv=5)
# Fit grid search
%time grid_search.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Pull results from grid search
results = pd.DataFrame(grid_search.cv_results_)
hm_data = results.pivot(index='param_C', 
                   columns='param_gamma',
                   values='mean_test_score')

# Plot heatmap
fix, ax = plt.subplots(figsize=(9, 6))

ax = sns.heatmap(hm_data,
                 annot=True,
                 cmap='RdBu',
                 linecolor='white')

ax.set_title('Hyperparameter Gridsearch', fontsize=16)
ax.set_ylabel('C Parameter', fontsize=12)
ax.set_xlabel('Gamma Parameter', fontsize=12)

plt.show()

In [ ]:
# Print best parameters and best score
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

### Best parameters: {'C': 100, 'gamma': 0.0001}
### Best cross-validation score: 1.00

# Plug best parameters into the model, train it, and test on held out data
best_svc = SVC(**grid_search.best_params_).fit(X_train, y_train)
best_svc.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = best_svc.predict(X_test)

print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))